In [14]:
!pip install adamp

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5999 sha256=aa031aa65eab7078372f0d63689421954a727b26e4222ea12a7fe51b54c5fc6e
  Stored in directory: /Users/chang/Library/Caches/pip/wheels/b4/c9/9e/460d14f9b1d7eae27cc813b31373d578fcda1914ef1a19f2b7
Successfully built adamp


In [22]:
import pandas as pd
import numpy as np
import os

import transformers
from transformers import AutoTokenizer, AdamW, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm, tqdm_notebook

import random
import torch.backends.cudnn as cudnn

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from adamp import AdamP

In [16]:
df = pd.read_excel('한국어_단발성_대화_데이터셋.xlsx')
df.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0


In [18]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,우승한다고하던첼시팬들다어디감,혐오,NaN,NaN,NaN,NaN,NaN
1,늘고마워요,행복,NaN,NaN,NaN,NaN,NaN
2,내 취향은 아니라고 하니 자기 눈에는 이쁘다며다른사람들 눈에도 괜찮을거라 하네요 ㅎㅎㅎ,행복,NaN,NaN,NaN,NaN,NaN
3,흥민아 라멜라랑 패날트킥 싸움 할필요없다. 다음엔 니가 차면된다..,중립,NaN,NaN,NaN,NaN,NaN
4,군납비리부터 겁나게비싸게사니,혐오,NaN,NaN,NaN,NaN,NaN


In [20]:
df_shuffled.loc[(df_shuffled['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
df_shuffled.loc[(df_shuffled['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
df_shuffled.loc[(df_shuffled['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
df_shuffled.loc[(df_shuffled['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
df_shuffled.loc[(df_shuffled['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
df_shuffled.loc[(df_shuffled['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
df_shuffled.loc[(df_shuffled['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [23]:
#train, test = train_test_split(df_shuffled, test_size=0.25, random_state=0)

In [76]:
train, val = train_test_split(train, test_size=0.2, random_state=2021)

In [77]:
train=train[['Sentence','Emotion']]
test=test[['Sentence']]

In [78]:
class NTDataset(Dataset):
  
    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

        print(self.dataset.describe())

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:3].values
        text = row[0]
        y = row[1]
        inputs = self.tokenizer(
            text, 
            return_tensors='pt',
            truncation=True,
            max_length=15,
            pad_to_max_length=True,
            add_special_tokens=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, y

In [79]:
class NTDataset_test(Dataset):
  
    def __init__(self, csv_file):
        self.dataset = csv_file
        self.tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large")

        print(self.dataset.describe())
  
    def __len__(self):
        return len(self.dataset)
  

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 1:2].values
        text = row[0]
        inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=15,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask

In [80]:
train_dataset = NTDataset(train)
test_dataset = NTDataset_test(test)

       Sentence  Emotion
count     23156    23156
unique    23120        7
top        미안해.        5
freq          3     3631
       Sentence
count      9649
unique     9642
top       안녕하세요
freq          2


In [81]:
device = torch.device("cpu")

In [82]:
model = RobertaForSequenceClassification.from_pretrained("klue/roberta-large", num_labels=7).to(device)

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

In [83]:
epochs = 3
batch_size = 128

In [84]:
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [85]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

IndexError: index 1 is out of bounds for axis 0 with size 1

In [57]:
# train
losses = []
accuracies = []
total_loss = 0.0
correct = 0
total = 0

for i in range(epochs):

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss / total, "Accuracy:", correct.float() / total)

  0%|          | 0/227 [00:00<?, ?it/s]

IndexError: index 1 is out of bounds for axis 0 with size 1